# Project test

In this assignment, we are going to improve our codes in PattRecClasses and implement forward algorithm inside MarkovChain code as well as functions such as logprob and prob in Guassian in order to generate proper input values for forward algorithm.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
from matplotlib import pyplot as plt

from CharacterFeatureExtractor import featureExtractor
from DrawCharacter import DrawCharacter
from PattRecClasses import HMM_TA 
from hmm_gen import hmm_gen

In [2]:
'''
Multivariate Gaussian Distribution Class
'''
class multigaussD:
    mean = np.array([0])
    cov = np.array([[0]])
    def __init__(self, mu, C):
        if C.shape[0] is not C.shape[1]:
            print("error, non-square covariance matrix supplied")
            return
        if mu.shape[0] is not C.shape[0]:
            print("error, mismatched mean vector and covariance matrix dimensions")
            return
        self.mean = mu
        if np.where(np.diag(C)==0)[0].shape[0] != 0:
            C += np.diagflat(np.ones(C.shape[0])/10000)
        C[np.isnan(C)]=1
        self.cov = C
        return
    def random(self, num):
        return np.random.multivariate_normal(self.mean, self.cov, num)
    def rand(self):
        return np.random.multivariate_normal(self.mean, self.cov, 1)[0]
    def likelihood(self, X):
        p = scipy.stats.multivariate_normal(self.mean, self.cov, 1)
        pd = p.pdf(X)
        return pd
    def loghood(self, X):
        return np.log(self.likelihood(X))
    def getmean(self):
        return self.mean
    def getcov(self):
        return self.cov
    
def prob(x, B):
    T = x.shape[0]
    N = B.shape[0]
    res = np.zeros((T, N))
    for i in range(T):
        for j in range(N):
            res[i,j] = B[j].likelihood(x[i])
    scaled = np.zeros(res.shape)
    for i in range(scaled.shape[0]):
        for j in range(scaled.shape[1]):
            scaled[i, j] = res[i,j]/np.amax(res[i])
    return res, scaled


def logprob(x, B):
    res, scaled = prob(x,B)
    return np.log(res), np.log(scaled)

# Training model

In [3]:
### data prep
db_name = "database_inc_sampchar"
data_features = pd.read_pickle(r'data/' + db_name + '_features.cdb')
data_labels = pd.read_pickle(r'data/' + db_name + '_labels.cdb')

# data_features[k][r] == np.array (ndim, t); K (number of letters) of R samples with Tr individual lengths
# print((data_features[1][1].shape))
print(data_labels)

# train for one character (seen from labels)

hm_learn = [0,0,0,0,0]

# Start by assigning initial HMM parameter values,
# then refine these iteratively

# A (char = 0)
# States = 3
qstar = np.array([1, 0, 0])
Astar = np.array([[0.9, 0.1, 0], [0, 0.9, 0.1], [0, 0, 1]])
meansstar = np.array([[10, 60], [10, -70], [40,0]])
covsstar = np.array([[[1, 1], [1, 1]],
                     [[1, 1], [1, 1]], [[1,1],[1,1]]])

print(covsstar)

Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1]), 
                  multigaussD(meansstar[2], covsstar[2])])

hm_learn[0] = HMM_TA.HMM(qstar, Astar, Bstar)

# C (char = 1)
# States = 5 ...

# X (char = 4)
# States = 2
qstar = np.array([1, 0])
Astar = np.array([[0.9, 0.1], [0, 1]])
meansstar = np.array([[10, -60], [45, 70]])
covsstar = np.array([[[1, 1], [1, 1]],
                     [[1, 1], [1, 1]]])
Bstar = np.array([multigaussD(meansstar[0], covsstar[0]),
                  multigaussD(meansstar[1], covsstar[1])])

print(meansstar[0])


hm_learn[4] = HMM_TA.HMM(qstar, Astar, Bstar)

hm_learn[1] = hm_learn[0]
hm_learn[2] = hm_learn[0]
hm_learn[3] = hm_learn[0]

['A', 'C', 'K', 'P', 'X', 'T', '+', 'N', 'V', '4']
[[[1 1]
  [1 1]]

 [[1 1]
  [1 1]]

 [[1 1]
  [1 1]]]
[ 10 -60]


In [4]:
""" hmm_learned, training_accuracy =  hmm_train(train_data, labels)


"""
hm_learn = hmm_gen(data_features, 10, print=False)

train_data = []
test_data = []
train_results = []

for char in range(3):  # len(data_features)
    print("\n")
    print("------------ CHARACTER", data_labels[char], "------------")
    # Data preprocessing 
    obs = data_features[char]
    # oos_obs = data_features[char-1]
    
    # obsTA = np.array([ hm_learn.rand(100)[0] for _ in range(10) ])
    # print(type(obsTA))
    # print(obsTA[1].shape) == (100,2)
    # Our data has format (2,15) ! Transpose all datapoints
    for i in range(len(obs)):
        obs[i] = np.transpose(obs[i])
    
    #data_features[char] = obs  # so we do not have to reinvert the data later
    
    #for i in range(len(oos_obs)):
    #    oos_obs[i] = np.transpose(oos_obs[i])

    # Data information
    """
    print(len(obs))
    print(obs[len(obs) - 1].shape)
    print(type(obs))
    print(obs[1])
    """

    # Divide data into training and testing
    
    train_obs = obs[0:len(obs)-5]
    test_obs = obs[len(obs)-5:len(obs)]
    
    print("Training data", len(train_obs))
    print("Testing data", len(test_obs))
    
    # Training
    print("Running the Baum Welch Algorithm...")
    hm_learn[char].baum_welch(train_obs, 20, prin=1, uselog=False)

    
    # Testing on out of sample and test obs
    
    #a, c = hm_learn[char].alphahat(oos_obs[2])
    #print("Prob oos", c)
    lprob_list = []
    
    
    # Testing! 
    
    for test in test_obs:
        a, c = hm_learn[char].alphahat(test)
        print("c is", c)
        clog = np.log(c)
        lprob = np.sum(np.array(clog))
        lprob_list += [lprob]
        
    print(lprob_list)
    
    avg = np.mean(np.array(lprob_list))
    print("Avg probability over test samples is", np.exp(avg))
    train_results += [avg]


10


AttributeError: 'HMM' object has no attribute 'shape'